In [19]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
%autoreload 2

In [21]:
import functools
import jax
import os
import pandas as pd
import mlflow
import pickle

from datetime import datetime
from jax import numpy as jnp
import matplotlib.pyplot as plt

from IPython.display import HTML, clear_output
import mediapy

import brax

import flax
from brax import envs
from brax.io import model
from brax.io import json
from brax.io import html

# other envs
from task_aware_skill_composition.brax.envs.point import Point
from task_aware_skill_composition.brax.envs.car import Car
from task_aware_skill_composition.brax.envs.drone import Drone
from task_aware_skill_composition.brax.envs.point import Point
from task_aware_skill_composition.brax.envs.doggo import Doggo
from jaxgcrl.envs.ant_maze import AntMaze

# tasks
from task_aware_skill_composition.brax.tasks import get_task

In [31]:
backend = 'mjx'

# env = Car(backend=backend)
# env = Drone(backend=backend)
# env = Point(backend=backend)
# env = Doggo(backend=backend)

# env = envs.get_environment(env_name="reacher", backend=backend)
# env = AntMaze(backend=backend)

# task = get_task("point", "one_goal")
task = get_task("xy_point", "turn_seq")
env = task.env

spec = task.lo_spec
spec_tag = type(task).__name__
env_tag = type(env).__name__

jit_reset = jax.jit(env.reset)
jit_step = jax.jit(env.step)

# reset the environment
rng = jax.random.PRNGKey(2)
state = jit_reset(rng)

# grab a trajectory acting according to constant ctrl
ctrl1 = jnp.array([0.5, 0.5])
ctrl2 = jnp.array([0.5, 0.00])
ctrl3 = jnp.array([0.00, 0.00])

rollout = [state]
actions = []
n_steps = 200
render_every = 1

for i in range(n_steps):
    act_rng, rng = jax.random.split(rng)
    # ctrl, _ = jit_inference_fn(state.obs, act_rng)
    if i < n_steps//3:
        ctrl = ctrl1
    elif i < 2*n_steps//3:
        ctrl = ctrl2
    else:
        ctrl = ctrl3
    
    state = jit_step(state, ctrl)

    if state.done:
        break

    # print(type(state.pipeline_state))

    actions.append(ctrl)
    rollout.append(state)

# mediapy.show_video(
#     env.render(
#         [s.pipeline_state frictionloss="0.2"for s in rollout],
#         camera='track'
#     ), fps=1.0 / env.dt
# )

In [32]:
HTML(html.render(env.sys, [state.pipeline_state for state in rollout]))

# With Previous Run